# 소프트맥스 회귀
- 선형 회귀의 한계점
> - 비선형 데이터의 경우 적절한 예측을 할 수 없다.
> - 이상치 데이터에 대해서 취약하다.
- 로지스틱 회귀의 한계점
> - 클래스 간 경계선이 비선형인 경우 분류 성능이 저하됨
> - 클래스 간 데이터가 크게 불균형한 경우 분류 성능이 저하됨

Softmax regression은 로지스틱 회귀의 일반화된 형태로, 다중 클래스 분류 문제에서 사용되는 알고리즘이다. 목적은 각 클래스에 대한 확률 출력.<br>
소프트맥스 회귀는 입력 변수(x)와 출력 변수(y) 사이의 선형 관계를 모델링하며, 선형 회귀와 유사한 방법을 사용한다. 다만, 선형 회귀는 예측값을 연속적인 값으로 출력하는 반면, 소프트맥스 회귀는 각 클래스에 대한 확률 갑슬 출력한다.<br>
모델은 입력변수(x)와 가중치(w)의 곱의 합에 편향(b)을 더한 값에 대해 소프트맥스 함수를 적용한다. 소프트맥스 함수는 각 클래스에 대한 확률 값을 출력하기 위해 모델의 출력 값을 0과 1 사이의 값으로 변환한다. 이렇게 변환된 값은 모든 클래스의 출력 값 합이 1이 되도록 정규화한다. 따라서, 각 클래스에 대한 확률 값은 소프트맥스 함수의 출력 값으로 계산된다.<br>
softmax regression은 다중 클래스 분류 문제에서 효과적으로 사용됨

- Softmax function의 정의
> softmax(zj) = exp(z_j)/sum_{k=1}^{K}\exp(z_k)
>> z : 입력 벡터, j : 클래스 인덱스, 분모 : 입력 벡터의 모든 원소에 대해 지수 함수를 취한 값의 합, exp(zj) : 입력 벡터의 j번째 원소에 대한 지수 함수 값 의미
> 소프트맥스 함수는 입력 벡터의 모든 원소를 대상으로 계산하며, 각 클래스의 확률값을 계산함
> 소프트맥스 함수의 이점 중 하나는 각 클래스에 대한 확률값이 항상 0과 1 사이의 값으로 나오며, 이들의 총합은 1이 된다는 것이다.
> 이는 다중 클래스 분류 문제에서 각 클래스에 대한 예측 확률을 쉽게 해석할 수 있도록 해준다. 또한, 소프트맥스 함수는 출력값이 확률 분포를 따르기 때문에, 분류 모델에서 정확도를 최적화하는 목적 함수로 사용된다.

- 소프트맥스 함수의 비용 함수
크로스 엔트로피 함수. 이 함수는 모델의 예측값과 실제값 사이의 차이를 측정하여 모델의 성능을 평가하는 데 사용된다. 따라서 예측값이 실제값과 일치하도록 학습하는 것을 목적으로 한다. 이 함수의 값이 작을수록 모델의 예측이 실제값과 가까워진다.

### Pytorch를 사용하여 소프트맥스 회귀 모델을 학습하고 예측하는 코드
iris 데이터셋을 사용하여 소프트맥스 회귀 모델을 학습하고 테스트 실습

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load iris dataset
iris = load_iris()
x = iris.data
y = iris.target

# Split dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Convert data to Pytorch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

# Create a Pytorch DataLoader object for the training set
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# Define the softmax regression model
class SoftmaxRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        return out
    
# Set the hyperparameters
input_size = 4
num_classes = 3
lr = 0.01
num_epochs = 100

# Create the softmax regression model and optimizer
model = SoftmaxRegression(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward Pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print the loss every 10 batches
        if (i+1) % 10 == 0:
            print("Epoch [{}/{}], step[{}/{}], Loss : {:.4f}" .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
# Evaluate the model on the testing set
with torch.no_grad():
    outputs = model(x_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print('Test Accuracy : {:.2f}%'.format(accuracy * 100))

Epoch [1/100], step[10/12], Loss : 0.6910
Epoch [2/100], step[10/12], Loss : 0.8725
Epoch [3/100], step[10/12], Loss : 0.8317
Epoch [4/100], step[10/12], Loss : 0.7932
Epoch [5/100], step[10/12], Loss : 0.7771
Epoch [6/100], step[10/12], Loss : 0.6729
Epoch [7/100], step[10/12], Loss : 0.6727
Epoch [8/100], step[10/12], Loss : 0.6346
Epoch [9/100], step[10/12], Loss : 0.5910
Epoch [10/100], step[10/12], Loss : 0.6798
Epoch [11/100], step[10/12], Loss : 0.5145
Epoch [12/100], step[10/12], Loss : 0.6254
Epoch [13/100], step[10/12], Loss : 0.5856
Epoch [14/100], step[10/12], Loss : 0.4794
Epoch [15/100], step[10/12], Loss : 0.6746
Epoch [16/100], step[10/12], Loss : 0.4515
Epoch [17/100], step[10/12], Loss : 0.5169
Epoch [18/100], step[10/12], Loss : 0.6148
Epoch [19/100], step[10/12], Loss : 0.6684
Epoch [20/100], step[10/12], Loss : 0.6556
Epoch [21/100], step[10/12], Loss : 0.6111
Epoch [22/100], step[10/12], Loss : 0.4140
Epoch [23/100], step[10/12], Loss : 0.5731
Epoch [24/100], step

# Softmax Regression 모델이 학습한 결정 경계와 데이터 포인트 시각화

In [8]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Create toy dataset
x, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=42)

# Convert data to Pytorch tensors
x = torch.from_numpy(x).float()
y = torch.from_numpy(y)

# Define the softmax regression model
class SoftmaxRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        return out
    
# Instantiate model
input_size = 2
num_classes = 3
model = SoftmaxRegression(input_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)


# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward Pass amd calculate loss
    outputs = model(x)
    loss = criterion(outputs, y)
        
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    # Print the loss every 10 batches
    if (epoch+1) % 100 == 0:
        print("Epoch [{}/{}], Loss : {:.4f}" .format(epoch+1, num_epochs, loss.item()))

# Plot decision boundary
x_min, x_max = x[:, 0].min() - 0.5, x[:, 0].max() + 0.5
y_min, y_max = x[:, 1].min() - 0.5, x[:, 1].max() + 0.5
xx, yy = torch.meshgrid(torch.arange(x_min, x_max, 0.1), torch.arange(y_min, y_max, 0.1))
z = model(torch.cat((xx.reshape(-1, 1), yy.reshape(-1, 1)), dim=1)).argmax(dim=1)
z = z.reshape(xx.shape)
plt.contourf(xx, yy, z, alpha=0.4)
plt.scatter(x[:, 0], x[:, 1], c=y, s=20, edgecolors='k')
plt.title('Softmax Regression')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

RuntimeError: expected scalar type Long but found Int

# 위 코드는 다시 체크 필요

## 소프트맥스 회귀의 한계
- 과적합 : 매개변수의 수가 많은 모델은 데이터를 잘 설명할 수 있지만, 일반화 능력이 떨어지는 과적합 문제가 발생할 수 있다.
- 클래스 수에 따른 모델 복잡도 증가 : 클래스 수가 증가할수록 모델의 복잡도가 증가하여, 학습 데이터가 적을 때 모델의 성능이 저하될 수 있다.
- 이진 분류만 가능 : 소프트맥스 회귀는 이진 분류만 가능하며, 다중 분류에 대한 처리를 위해서는 One-vs-Rest 또는 One-vs-One 방법을 사용해야 한다.
## 소프트맥스 회귀의 한계 극복 방법
- 정규화 : L1 또는 L2 정규화 등의 방법 사용
- 드롭 아웃 : 일부 매개변수를 무작위로 제거해 모델의 일반화 능력 향상
- 다중 레이블 분류
- 다른 분류 모델과 결합
- 다른 확률 분포 모델과 결합